In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

# Direct path specification
env_path = Path('../backend/.env').resolve()  # or Path('../backend/.env') depending on your directory structure

# Load environment variables from .env file with explicit path
load_dotenv(dotenv_path=env_path)

# Access your API key
api_key = os.getenv("EMBER_API_KEY")

# Verify it loaded (don't print the actual key in your notebook!)
print("API key loaded:", "✓" if api_key else "✗")
print(f"Looking for .env file at: {env_path}")

API key loaded: ✓
Looking for .env file at: /Users/allison/workspace/ai playground/steering-interface/backend/.env


In [2]:
import goodfire

In [3]:
client = goodfire.AsyncClient(
    api_key=api_key,
)

In [13]:
async def chat_completion(messages: list[dict], model: goodfire.Variant):
    response = await client.chat.completions.create(
        messages=messages,
        model=model
    )
    print(response.choices[0].message["content"])

await chat_completion([{"role": "user", "content": "What is the capital of France?"}], vDefault)

The capital of France is Paris.


In [14]:
# Create a variant

vDefault = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")
print(vDefault)

Variant(
   base_model=meta-llama/Llama-3.3-70B-Instruct,
   edits={
   }
   scopes={
   }
)


In [27]:
# Search features
async def search_features(query: str, model: goodfire.Variant):
    features = await client.features.search(query, model=model, top_k=10)
    feature = features[0]
    print(feature.uuid)
    print(feature.label)
    print(feature.index_in_sae)

await search_features("ELI5", vDefault)

b2e24a95-f194-4994-9d39-b48efd142e7d
Simplified explanations of complex concepts
22933


In [28]:
async def list_features(feature_ids: list[str]):
    """List features by their UUIDs using async client."""
    try:
        # Use 'ids' parameter as shown in documentation
        features = await client.features._list(ids=feature_ids)
        print(f"Found {len(features)} features:")
        for feature in features:
            print(f"  - {feature.label} ({feature.uuid})")
        return features
    except AttributeError as e:
        print(f"Method not available: {e}")
        return None

await list_features(["b2e24a95-f194-4994-9d39-b48efd142e7d"])

Found 1 features:
  - Simplified explanations of complex concepts (b2e24a95-f194-4994-9d39-b48efd142e7d)


FeatureGroup([
   0: "Simplified explanations of complex concepts"
])

In [26]:
async def lookup_features(indices: list[int], model: goodfire.Variant):
    features = await client.features.lookup(indices=indices, model=model)
    print(features)

await lookup_features([0, 1, 2], vDefault)

{1: Feature("The assistant wishes enjoyment of food/recipes to users"), 2: Feature("Brazilian historical narratives in Portuguese, especially about colonial discoveries"), 0: Feature("Explanatory statements about AI's impacts and applications")}


In [20]:
# Debug: See what methods are available
print("Available methods on client.features:")
print([method for method in dir(client.features) if not method.startswith('_')])

Available methods on client.features:
['AutoConditional', 'AutoSteer', 'activations', 'base_url', 'contrast', 'generate_contrastive_stimulus', 'goodfire_api_key', 'inspect', 'lookup', 'neighbors', 'rerank', 'search', 'semantic_similarity']


In [10]:
inspector = client.features.inspect(
    [
        {"role": "user", "content": "What do you think about pirates and whales"},
        {"role": "assistant", "content": "I think pirates are cool and whales are cool"}
    ],
    model=vDefault
)

# Get top activated features
for activation in inspector.top(k=20):
    print(f"{activation.feature.label}: {activation.activation}")

AttributeError: 'coroutine' object has no attribute 'top'

In [8]:
# Compare formal vs informal conversations
dataset_1 = [[
    {"role": "user", "content": "Hi how are you?"},
    {"role": "assistant", "content": "I'm doing well..."}
]]
dataset_2 = [[
    {"role": "user", "content": "Hi how are you?"},
    {"role": "assistant", "content": "Arr my spirits be high..."}
]]

formal_features, informal_features =  client.features.contrast(
    dataset_1=dataset_1,
    dataset_2=dataset_2,
    model="meta-llama/Llama-3.3-70B-Instruct",
    top_k=5
)

In [12]:
formal_features
random_feature = formal_features[0]

In [10]:
# Rerank features based on relevance to "writing style"
reranked = client.features.rerank(
    features=formal_features,
    query="writing style",
    model="meta-llama/Llama-3.3-70B-Instruct",
    top_k=10
)

reranked

FeatureGroup([
   0: "Assistant responding to casual greetings about its wellbeing",
   1: "Status check questions and responses using 'doing' in conversation",
   2: "The assistant maintaining honesty about being an AI",
   3: "Assistant's initial friendly greeting to open a conversation",
   4: "Positive responses to 'how are you' questions across languages"
])

In [20]:
random_feature.uuid

'8bb53ed7-ed2f-4166-9161-13df6006451d'